# Neurons example via low-level, flexible interface
## Prepare

In [1]:
from bayes_window import models
from bayes_window.fitting import fit_numpyro
from bayes_window.generative_models import generate_fake_spikes
from sklearn.preprocessing import LabelEncoder

trans = LabelEncoder().fit_transform

## Make some data


In [2]:
df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=2,
                                                                n_neurons=8,
                                                                n_mice=4,
                                                                dur=7, )

In [3]:
import numpy as np
df['log_isi']=np.log10(df['isi'])

In [4]:
from bayes_window import visualization,utils
from importlib import reload
reload(visualization)
reload(utils)
y='isi'
df['neuron']=df['neuron'].astype(int)
ddf, dy = utils.make_fold_change(df,
                         y=y,
                         index_cols=('stim', 'mouse_code', 'neuron'),
                         condition_name='stim',
                         do_take_mean=True)

visualization.plot_data(x='neuron',y=dy, color='mouse_code',add_box=True,df=ddf)

alt.LayerChart(...)

## Estimate model

In [5]:
#y = list(set(df.columns) - set(index_cols))[0]
trace = fit_numpyro(y=df[y].values,
                    stim_on=(df['stim']).astype(int).values,
                    treat=trans(df['neuron']),
                    subject=trans(df['mouse']),
                    progress_bar=True,
                    model=models.model_hier_normal_stim,
                    n_draws=100, num_chains=1, )

sample: 100%|██████████| 1100/1100 [00:06<00:00, 166.41it/s, 15 steps of size 2.38e-01. acc. prob=0.77]


n(Divergences) = 0


## Add data back

In [6]:
reload(utils)
df_both = utils.add_data_to_posterior(df,
                                trace=trace,
                                y=y,
                                index_cols=['neuron', 'stim', 'mouse_code', ],
                                condition_name='stim',
                                b_name='b_stim_per_condition',  # for posterior
                                group_name='neuron'  # for posterior
                                )

## Plot data and posterior

In [7]:
#BayesWindow.plot_posteriors_slopes(df_both, y=f'{y} diff', x='neuron',color='mouse_code',title=y,hold_for_facet=False,add_box=False)


chart_d = visualization.plot_data(df=df_both,x='neuron', y=f'{y} diff',color='mouse_code')
chart_d

alt.LayerChart(...)

In [8]:
chart_p = visualization.plot_posterior(df=df_both, title=f'd_{y}', x='neuron',)
chart_p

alt.LayerChart(...)

In [9]:
(chart_d+chart_p).resolve_scale(y='independent')

alt.LayerChart(...)

In [10]:
(chart_d+chart_p).facet(column='neuron')

alt.FacetChart(...)